In [ ]:
#install requirements
!pip install tensorflow
!pip install numpy
!pip install pandas
!pip install opencv-python
!pip install Keras
!pip install matplotlib


2358.36s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Found existing installation: tensorflow 2.16.2
Uninstalling tensorflow-2.16.2:
  Successfully uninstalled tensorflow-2.16.2
Found existing installation: tensorflow-macos 2.16.2
Uninstalling tensorflow-macos-2.16.2:
  Successfully uninstalled tensorflow-macos-2.16.2
Found existing installation: tensorflow-metal 1.2.0
Uninstalling tensorflow-metal-1.2.0:
  Successfully uninstalled tensorflow-metal-1.2.0


2365.15s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


  Using cached tensorflow_macos-2.16.2-cp312-cp312-macosx_12_0_arm64.whl.metadata (3.3 kB)
  Using cached tensorflow-2.16.2-cp312-cp312-macosx_12_0_arm64.whl.metadata (4.1 kB)
Using cached tensorflow_macos-2.16.2-cp312-cp312-macosx_12_0_arm64.whl (2.1 kB)
Using cached tensorflow-2.16.2-cp312-cp312-macosx_12_0_arm64.whl (227.1 MB)


2377.37s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


...

2388.85s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


2394.64s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


2400.44s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


2406.22s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


  Using cached tensorflow_metal-1.2.0-cp312-cp312-macosx_12_0_arm64.whl.metadata (1.3 kB)
Using cached tensorflow_metal-1.2.0-cp312-cp312-macosx_12_0_arm64.whl (1.4 MB)


In [ ]:
import tensorflow as tf
print("TensorFlow版本:", tf.__version__)
print("可用设备:", tf.config.list_physical_devices())

TensorFlow版本: 2.16.2
可用设备: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
# Imports
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
from pathlib import Path
import os
from sklearn.model_selection import train_test_split
from keras import models, layers


In [ ]:
# 加载训练集和验证集（位于不同目录）

# 训练集路径
train_base_dir = Path('./FTC Sample Detection.v2i.tensorflow/train')
train_annotations = pd.read_csv(train_base_dir / '_annotations.csv')
train_total_rows = len(train_annotations)
print(f"训练集CSV包含 {train_total_rows} 条记录")

# 验证集路径 - 请替换为你的实际验证集路径
val_base_dir = Path('./FTC Sample Detection.v2i.tensorflow/valid') # 根据实际路径修改
val_annotations = pd.read_csv(val_base_dir / '_annotations.csv')
val_total_rows = len(val_annotations)
print(f"验证集CSV包含 {val_total_rows} 条记录")

# 类别映射
class_mapping = {
    'blue sample': 0,
    'red sample': 1,
    'yellow sample': 2
}

# 处理数据集的函数
def process_dataset(annotations, base_dir, class_mapping):
    valid_entries = []
    total_rows = len(annotations)

    for i, row in annotations.iterrows():
        image_path = base_dir / row['filename']
        class_name = row['class']
        class_id = class_mapping.get(class_name, -1)

        if os.path.exists(image_path) and class_id != -1:
            # 创建数据条目
            entry = {
                'image_path': str(image_path),
                'class_id': class_id,
                'bbox': [
                    row['xmin'] / row['width'],
                    row['ymin'] / row['height'],
                    row['xmax'] / row['width'],
                    row['ymax'] / row['height']
                ],
                'width': row['width'],
                'height': row['height']
            }
            valid_entries.append(entry)

        # 进度显示
        if (i+1) % 100 == 0:
            print(f"处理了 {i+1}/{total_rows} 条记录")

    print(f"有效条目: {len(valid_entries)}/{total_rows}")
    return valid_entries

# 处理训练集和验证集
train_entries = process_dataset(train_annotations, train_base_dir, class_mapping)
val_entries = process_dataset(val_annotations, val_base_dir, class_mapping)

# 解析函数
def parse_function(entry):
    # 读取图像
    image_path = entry['image_path']
    img_raw = tf.io.read_file(image_path)
    image = tf.io.decode_image(img_raw, channels=3, expand_animations=False)
    image = tf.image.convert_image_dtype(image, tf.float32)  # 归一化到 [0,1]

    # 提取标签和边界框
    label = entry['class_id']
    bbox = entry['bbox']

    return image, (label, bbox)

# 创建训练集
train_dataset = tf.data.Dataset.from_generator(
    lambda: train_entries,
    output_signature={
        'image_path': tf.TensorSpec(shape=(), dtype=tf.string),
        'class_id': tf.TensorSpec(shape=(), dtype=tf.int32),
        'bbox': tf.TensorSpec(shape=(4,), dtype=tf.float32),
        'width': tf.TensorSpec(shape=(), dtype=tf.int32),
        'height': tf.TensorSpec(shape=(), dtype=tf.int32)
    }
)
train_dataset = train_dataset.map(parse_function, num_parallel_calls=tf.data.AUTOTUNE)

# 创建验证集
val_dataset = tf.data.Dataset.from_generator(
    lambda: val_entries,
    output_signature={
        'image_path': tf.TensorSpec(shape=(), dtype=tf.string),
        'class_id': tf.TensorSpec(shape=(), dtype=tf.int32),
        'bbox': tf.TensorSpec(shape=(4,), dtype=tf.float32),
        'width': tf.TensorSpec(shape=(), dtype=tf.int32),
        'height': tf.TensorSpec(shape=(), dtype=tf.int32)
    }
)
val_dataset = val_dataset.map(parse_function, num_parallel_calls=tf.data.AUTOTUNE)

# 批处理、预取优化
train_dataset = train_dataset.shuffle(buffer_size=len(train_entries)).batch(32).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

print(f"训练数据集大小: {len(train_entries)}")
print(f"验证数据集大小: {len(val_entries)}")

# 检查数据集
print("\n训练数据集示例:")
for images, (labels, bboxes) in train_dataset.take(1):
    print(f"批次大小: {images.shape[0]}")
    print(f"图像形状: {images.shape}")
    print(f"标签示例: {labels[:3].numpy()}")
    print(f"边界框示例: {bboxes[:3].numpy()}")

print("\n验证数据集示例:")
for images, (labels, bboxes) in val_dataset.take(1):
    print(f"批次大小: {images.shape[0]}")
    print(f"图像形状: {images.shape}")
    print(f"标签示例: {labels[:3].numpy()}")
    print(f"边界框示例: {bboxes[:3].numpy()}")

# 预处理数据集
def preprocess_dataset(dataset, input_shape=(224, 224)):
    def resize_image(image, label_bbox):
        image = tf.image.resize(image, input_shape)
        return image, label_bbox

    return dataset.map(resize_image, num_parallel_calls=tf.data.AUTOTUNE)

# 应用预处理
train_dataset = preprocess_dataset(train_dataset)
val_dataset = preprocess_dataset(val_dataset)

训练集CSV包含 3378 条记录
验证集CSV包含 307 条记录
处理了 100/3378 条记录
处理了 200/3378 条记录
处理了 300/3378 条记录
处理了 400/3378 条记录
处理了 500/3378 条记录
处理了 600/3378 条记录
处理了 700/3378 条记录
处理了 800/3378 条记录
处理了 900/3378 条记录
处理了 1000/3378 条记录
处理了 1100/3378 条记录
处理了 1200/3378 条记录
处理了 1300/3378 条记录
处理了 1400/3378 条记录
处理了 1500/3378 条记录
处理了 1600/3378 条记录
处理了 1700/3378 条记录
处理了 1800/3378 条记录
处理了 1900/3378 条记录
处理了 2000/3378 条记录
处理了 2100/3378 条记录
处理了 2200/3378 条记录
处理了 2300/3378 条记录
处理了 2400/3378 条记录
处理了 2500/3378 条记录
处理了 2600/3378 条记录
处理了 2700/3378 条记录
处理了 2800/3378 条记录
处理了 2900/3378 条记录
处理了 3000/3378 条记录
处理了 3100/3378 条记录
处理了 3200/3378 条记录
处理了 3300/3378 条记录
有效条目: 3378/3378
处理了 100/307 条记录
处理了 200/307 条记录
处理了 300/307 条记录
有效条目: 307/307


2025-03-22 23:51:48.527979: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Max
2025-03-22 23:51:48.528030: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2025-03-22 23:51:48.528034: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2025-03-22 23:51:48.528077: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-03-22 23:51:48.528097: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


训练数据集大小: 3378
验证数据集大小: 307

训练数据集示例:
批次大小: 32
图像形状: (32, 640, 640, 3)
标签示例: [0 1 0]
边界框示例: [[0.428125  0.390625  0.6140625 0.6234375]
 [0.1703125 0.3546875 0.278125  0.615625 ]
 [0.4       0.3640625 0.603125  0.578125 ]]

验证数据集示例:
批次大小: 32
图像形状: (32, 640, 640, 3)
标签示例: [0 2 1]
边界框示例: [[0.475     0.1546875 0.6296875 0.3625   ]
 [0.3546875 0.509375  0.584375  0.7671875]
 [0.575     0.396875  0.753125  0.690625 ]]


2025-03-22 23:51:51.690629: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-03-22 23:51:51.771225: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [ ]:
# Framework

def create_detection_model(input_shape=(224,224,3),num_classes=3):

    base_model = tf.keras.applications.MobileNetV2(
        input_shape=input_shape,
        include_top=False,
        weights='imagenet'
    )

    for layer in base_model.layers[:100]:
        layer.trainable = False

    inputs = tf.keras.Input(shape=input_shape)
    x = tf.keras.applications.mobilenet_v2.preprocess_input(inputs*255.0)
    x = base_model(x,training = False)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(256,activation='relu')(x)

    classification_output = layers.Dense(num_classes,activation="softmax",name = "classification")(x)
    bbox_output = layers.Dense(4,name='bbox_regression')(x)
    model = models.Model(inputs=inputs,outputs=[classification_output,bbox_output])

    return model

# Compile
def bbox_loss(y_true,y_pred):
    return tf.keras.losses.Huber()(y_true,y_pred)

input_shape = (224,224,3)
model = create_detection_model(input_shape=input_shape,num_classes=3)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss={
        'classification': 'sparse_categorical_crossentropy',  # 这是正确的
        'bbox_regression': 'mse'  # 边界框用均方误差损失
    },
    metrics={
        'classification': 'accuracy',
        'bbox_regression': tf.keras.metrics.MeanAbsoluteError()
    }
)

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply (Multiply) │ (None, 224, 224,  │          0 │ input_layer_1[0]… │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ true_divide         │ (None, 224, 224,  │          0 │ multiply[0][0]    │
│ (TrueDivide)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ subtract (Subtract) │ (None, 224, 224,  │          0 │ true_divide[0][0] │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mobilenetv2_1.00_2… │ (None, 7, 7,      │  2,257,984 │ subtract[0][0]    │
│ (Functional)        │ 1280)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 1280)      │          0 │ mobilenetv2_1.00… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 1280)      │          0 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │    327,936 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ classification      │ (None, 3)         │        771 │ dense[0][0]       │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bbox_regression     │ (None, 4)         │      1,028 │ dense[0][0]       │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,587,719 (9.87 MB)

 Trainable params: 2,191,175 (8.36 MB)

 Non-trainable params: 396,544 (1.51 MB)

In [ ]:

# preprocess dataset
def preprocess_dataset(dataset,input_shape=(224,224)):
    def resize_image(image,label_bbox):
        image = tf.image.resize(image,input_shape)
        return image,label_bbox

    return dataset.map(resize_image,num_parallel_calls=tf.data.AUTOTUNE)

train_dataset = preprocess_dataset(train_dataset)
val_dataset = preprocess_dataset(val_dataset)

In [ ]:
# 自定义训练循环
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
num_epochs = 30

# 准备TensorBoard
import os
import datetime  # 确保导入了datetime模块
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_summary_writer = tf.summary.create_file_writer(os.path.join(log_dir, 'train'))
val_summary_writer = tf.summary.create_file_writer(os.path.join(log_dir, 'validation'))

# 训练步骤
@tf.function
def train_step(images, labels, bboxes):
    with tf.GradientTape() as tape:
        # 前向传播
        class_preds, bbox_preds = model(images, training=True)

        # 计算损失
        class_loss = tf.keras.losses.sparse_categorical_crossentropy(labels, class_preds)
        # 使用tf.keras.losses.MSE而不是mean_squared_error
        bbox_loss = tf.keras.losses.MSE(bboxes, bbox_preds)

        # 总损失
        total_loss = tf.reduce_mean(class_loss) + tf.reduce_mean(bbox_loss)

    # 计算梯度
    gradients = tape.gradient(total_loss, model.trainable_variables)
    # 应用梯度
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return total_loss, tf.reduce_mean(class_loss), tf.reduce_mean(bbox_loss)

# 验证步骤
@tf.function
def val_step(images, labels, bboxes):
    # 前向传播
    class_preds, bbox_preds = model(images, training=False)

    # 计算损失
    class_loss = tf.keras.losses.sparse_categorical_crossentropy(labels, class_preds)
    # 使用tf.keras.losses.MSE而不是mean_squared_error
    bbox_loss = tf.keras.losses.MSE(bboxes, bbox_preds)

    # 总损失
    total_loss = tf.reduce_mean(class_loss) + tf.reduce_mean(bbox_loss)

    # 计算准确率
    class_accuracy = tf.reduce_mean(
        tf.cast(tf.equal(tf.argmax(class_preds, axis=1), tf.cast(labels, tf.int64)), tf.float32)
    )

    return total_loss, tf.reduce_mean(class_loss), tf.reduce_mean(bbox_loss), class_accuracy

best_val_loss = float('inf')
history = {
    'loss': [], 'class_loss': [], 'bbox_loss': [],
    'val_loss': [], 'val_class_loss': [], 'val_bbox_loss': [], 'val_class_accuracy': []
}

for epoch in range(num_epochs):
    # 训练
    train_loss = []
    train_class_loss = []
    train_bbox_loss = []

    for images, (labels, bboxes) in train_dataset:
        total_loss, class_loss, bbox_loss = train_step(images, labels, bboxes)
        train_loss.append(total_loss)
        train_class_loss.append(class_loss)
        train_bbox_loss.append(bbox_loss)

    # 计算训练平均损失
    avg_train_loss = tf.reduce_mean(train_loss)
    avg_train_class_loss = tf.reduce_mean(train_class_loss)
    avg_train_bbox_loss = tf.reduce_mean(train_bbox_loss)

    # 验证
    val_loss = []
    val_class_loss = []
    val_bbox_loss = []
    val_class_accuracy = []

    for val_images, (val_labels, val_bboxes) in val_dataset:
        total_loss, class_loss, bbox_loss, class_accuracy = val_step(val_images, val_labels, val_bboxes)
        val_loss.append(total_loss)
        val_class_loss.append(class_loss)
        val_bbox_loss.append(bbox_loss)
        val_class_accuracy.append(class_accuracy)

    # 计算验证平均损失
    avg_val_loss = tf.reduce_mean(val_loss)
    avg_val_class_loss = tf.reduce_mean(val_class_loss)
    avg_val_bbox_loss = tf.reduce_mean(val_bbox_loss)
    avg_val_class_accuracy = tf.reduce_mean(val_class_accuracy)

    # 更新历史
    history['loss'].append(avg_train_loss.numpy())
    history['class_loss'].append(avg_train_class_loss.numpy())
    history['bbox_loss'].append(avg_train_bbox_loss.numpy())
    history['val_loss'].append(avg_val_loss.numpy())
    history['val_class_loss'].append(avg_val_class_loss.numpy())
    history['val_bbox_loss'].append(avg_val_bbox_loss.numpy())
    history['val_class_accuracy'].append(avg_val_class_accuracy.numpy())

    # 保存最佳模型
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        model.save('best_model.keras')
        print(f"保存最佳模型，验证损失: {best_val_loss:.4f}")

    # 记录到TensorBoard
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', avg_train_loss, step=epoch)
        tf.summary.scalar('class_loss', avg_train_class_loss, step=epoch)
        tf.summary.scalar('bbox_loss', avg_train_bbox_loss, step=epoch)

    with val_summary_writer.as_default():
        tf.summary.scalar('loss', avg_val_loss, step=epoch)
        tf.summary.scalar('class_loss', avg_val_class_loss, step=epoch)
        tf.summary.scalar('bbox_loss', avg_val_bbox_loss, step=epoch)
        tf.summary.scalar('class_accuracy', avg_val_class_accuracy, step=epoch)

    # 打印进度
    print(f"Epoch {epoch+1}/{num_epochs} - "
          f"Loss: {avg_train_loss:.4f}, Class Loss: {avg_train_class_loss:.4f}, Bbox Loss: {avg_train_bbox_loss:.4f} - "
          f"Val Loss: {avg_val_loss:.4f}, Val Class Loss: {avg_val_class_loss:.4f}, "
          f"Val Bbox Loss: {avg_val_bbox_loss:.4f}, Val Class Acc: {avg_val_class_accuracy:.4f}")

print(f"\n要查看TensorBoard，请在终端运行: tensorboard --logdir={log_dir}")

2025-03-22 23:52:11.561526: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-03-22 23:52:11.579852: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Failed to deserialize the `graph_buf`.
2025-03-22 23:52:54.250360: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-03-22 23:53:00.815990: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


保存最佳模型，验证损失: 9.8701
Epoch 1/30 - Loss: 0.6946, Class Loss: 0.5984, Bbox Loss: 0.0963 - Val Loss: 9.8701, Val Class Loss: 2.4998, Val Bbox Loss: 7.3704, Val Class Acc: 0.4347


2025-03-22 23:53:25.184176: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-03-22 23:53:25.955544: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


保存最佳模型，验证损失: 4.6962
Epoch 2/30 - Loss: 0.5679, Class Loss: 0.5289, Bbox Loss: 0.0390 - Val Loss: 4.6962, Val Class Loss: 2.4227, Val Bbox Loss: 2.2736, Val Class Acc: 0.3285


2025-03-22 23:53:50.240626: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-03-22 23:53:51.030481: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


...

保存最佳模型，验证损失: 0.5467
Epoch 28/30 - Loss: 0.5088, Class Loss: 0.4919, Bbox Loss: 0.0169 - Val Loss: 0.5467, Val Class Loss: 0.5011, Val Bbox Loss: 0.0456, Val Class Acc: 0.7081


2025-03-23 00:03:59.742061: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-03-23 00:04:00.551477: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 29/30 - Loss: 0.5085, Class Loss: 0.4918, Bbox Loss: 0.0167 - Val Loss: 0.5578, Val Class Loss: 0.5087, Val Bbox Loss: 0.0490, Val Class Acc: 0.7081


2025-03-23 00:04:22.815294: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-03-23 00:04:23.576260: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


保存最佳模型，验证损失: 0.5427
Epoch 30/30 - Loss: 0.5101, Class Loss: 0.4937, Bbox Loss: 0.0164 - Val Loss: 0.5427, Val Class Loss: 0.4868, Val Bbox Loss: 0.0558, Val Class Acc: 0.7049

要查看TensorBoard，请在终端运行: tensorboard --logdir=logs/fit/20250322-235206


In [ ]:
# 使用已保存的权重和重建模型创建TFLite模型

import tensorflow as tf
import numpy as np
import os

# 1. 首先定义和重建模型
def create_detection_model(input_shape=(224, 224, 3), num_classes=3):
    inputs = tf.keras.Input(shape=input_shape)
    base_model = tf.keras.applications.MobileNetV2(
        input_shape=input_shape,
        include_top=False,
        weights=None  # 不使用预训练权重
    )

    # 冻结基础模型的前几层
    for layer in base_model.layers[:100]:
        layer.trainable = False

    x = base_model(inputs, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)

    # 分类头
    classification_output = tf.keras.layers.Dense(num_classes, activation='softmax', name='classification')(x)

    # 边界框回归头
    bbox_output = tf.keras.layers.Dense(4, name='bbox_regression')(x)

    # 创建模型
    return tf.keras.Model(inputs=inputs, outputs=[classification_output, bbox_output])

# 2. 创建新模型实例
new_model = create_detection_model()

# 3. 从已保存的NumPy文件加载权重
weight_dir = "model_weights_numpy"
weights = []

# 按顺序加载权重
for i in range(266):  # 使用你打印的权重总数
    weight_file = os.path.join(weight_dir, f"weight_{i}.npy")
    if os.path.exists(weight_file):
        weights.append(np.load(weight_file))
    else:
        print(f"找不到权重文件: {weight_file}")
        break

# 检查是否所有权重都加载成功
if len(weights) == 266:
    # 4. 将权重应用到新模型
    try:
        new_model.set_weights(weights)
        print("成功将保存的权重应用到新模型")

        # 5. 使用最直接的方法转换为TFLite
        # 通过具体输入和输出创建转换器
        input_shape = [1, 224, 224, 3]  # 批次大小为1

        # 创建转换用的输入数据
        sample_input = np.random.random(input_shape).astype(np.float32)

        # 使用具体函数进行转换
        @tf.function
        def serving_fn(x):
            return new_model(x)

        concrete_func = serving_fn.get_concrete_function(
            tf.TensorSpec(input_shape, tf.float32))

        converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
        converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]

        try:
            # 标准精度转换
            tflite_model = converter.convert()
            with open("detector_model.tflite", "wb") as f:
                f.write(tflite_model)
            print("成功保存标准TFLite模型到 detector_model.tflite")

            # 尝试进行量化优化
            converter.optimizations = [tf.lite.Optimize.DEFAULT]
            try:
                optimized_tflite_model = converter.convert()
                with open("detector_model_optimized.tflite", "wb") as f:
                    f.write(optimized_tflite_model)
                print("成功保存优化TFLite模型到 detector_model_optimized.tflite")
            except Exception as e:
                print(f"优化TFLite模型失败: {e}")

        except Exception as convert_error:
            print(f"TFLite转换失败: {convert_error}")

            # 备选方法：更简单的转换方式
            try:
                # 确保输入形状正确
                dummy_input = tf.ones([1, 224, 224, 3], dtype=tf.float32)
                # 确保模型可以正确推理
                test_outputs = new_model(dummy_input)
                print(f"模型测试成功，输出形状: {[o.shape for o in test_outputs]}")

                # 尝试使用简单的转换方法
                simple_converter = tf.lite.TFLiteConverter.from_keras_model(new_model)
                simple_tflite_model = simple_converter.convert()

                with open("detector_model_simple.tflite", "wb") as f:
                    f.write(simple_tflite_model)
                print("成功使用简单方法保存TFLite模型到 detector_model_simple.tflite")
            except Exception as simple_error:
                print(f"简单TFLite转换也失败了: {simple_error}")
                print("请考虑使用以下方法直接在推理时加载权重:")
                print("""
                # 在推理代码中:
                def load_model_from_weights():
                    model = create_detection_model()
                    weights = []
                    for i in range(266):
                        weights.append(np.load(f"model_weights_numpy/weight_{i}.npy"))
                    model.set_weights(weights)
                    return model

                # 然后使用这个模型进行预测
                model = load_model_from_weights()
                predictions = model(image_batch)
                """)
    except Exception as e:
        print(f"应用权重到新模型失败: {e}")
else:
    print(f"无法加载所有权重，只找到 {len(weights)} 个权重文件，但需要 266 个")

模型文件存在，但格式可能不是SavedModel
尝试直接从内存中导出模型...


上述转换方法失败: 'Functional' object has no attribute '_get_save_spec'
模型已保存为H5格式到 model.h5
H5格式转换失败: Unknown layer: 'TrueDivide'. Please ensure you are using a `keras.utils.custom_object_scope` and that this object is included in the scope. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.
权重保存和具体函数转换失败: The filename must end in `.weights.h5`. Received: filepath=model_weights.h5
尝试重建模型...


重建的模型已保存到 rebuilt_model.h5
重建模型并保存失败: 'Functional' object has no attribute '_get_save_spec'
模型权重已保存为NumPy数组到目录: model_weights_numpy
权重总数: 266
